torch -> onnx -> tensorflow -> tflite

In [1]:
!pip install segmentation_models_pytorch

In [2]:
!cp -rf /content/drive/MyDrive/unetpp_4leaf.pth ./

In [3]:
import segmentation_models_pytorch as smp
import torch.onnx

In [4]:
onnx_model_path = './4leaf.onnx'
tf_model_path = './tf_onnx_model'
tflite_model_path = './4leaf.tflite'

# Load Torch

In [5]:
model = smp.UnetPlusPlus(
    encoder_name="efficientnet-b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
)

In [6]:
model.load_state_dict(torch.load('./unetpp_4leaf.pth', map_location='cpu'))

<All keys matched successfully>

# Torch to ONNX

In [7]:
!pip install onnx

In [8]:
import torch.onnx

dummy_input = torch.randn(1, 3, 544, 960, requires_grad=True)

torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
    export_params=True,
    opset_version=10,
    do_constant_folding=True,
    input_names = ['modelInput'],
    output_names = ['modelOutput']
)

/usr/local/lib/python3.10/dist-packages/segmentation_models_pytorch/base/model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:
/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_helper.py:833: UserWarning: You are trying to export the model with onnx:Resize for ONNX opset version 10. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend using opset 11 and above for models using this operator.
  warnings.warn(

# ONNX to Tensorflow

In [10]:
!git clone https://github.com/onnx/onnx-tensorflow.git
%cd onnx-tensorflow
!pip install -e .
%cd ../

fatal: destination path 'onnx-tensorflow' already exists and is not an empty directory.
/content/onnx-tensorflow
Obtaining file:///content/onnx-tensorflow
  Preparing metadata (setup.py) ... done
  Attempting uninstall: onnx-tf
    Found existing installation: onnx-tf 1.10.0
    Uninstalling onnx-tf-1.10.0:
      Successfully uninstalled onnx-tf-1.10.0
  Running setup.py develop for onnx-tf
/content


In [11]:
from onnx_tf.backend import prepare

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [13]:
import onnx

onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

In [14]:
tf_rep = prepare(onnx_model)

In [15]:
tf_rep.export_graph(tf_model_path)

In [16]:
import tensorflow as tf

model = tf.saved_model.load(tf_model_path)
model.trainable = False

input_tensor = tf.random.uniform([1, 3, 544, 960])
out = model(**{'modelInput': input_tensor})

# Tensorflow to TFLITE

In [17]:
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
tflite_model = converter.convert()

with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)